# Init

In [140]:
! pip install gamspy

In [141]:
! gamspy install license 204a1a5e-5e64-4583-a281-508165b2ee7e

In [142]:
import pandas as pd
from gamspy import Container, Set, Variable, Parameter, Equation, Sum, Model, Sense, Alias

# Data

In [154]:
# Load the CSV file into csv_data
csv_data = pd.read_csv("edge_distances_osrm.csv")

# Display the first few rows of the DataFrame
csv_data

,edge_name,distance_m,duration_s,from_node,to_node,distance_km,duration_min,avg_speed_kmh,u_ij,a_ij
0,E18 North,9333.6,628.5,Täby,Bergshamra,9.3336,10.475000,53.462148,2000,3.182180
1,E18 South,8989.9,704.5,Bergshamra,T-Centralen,8.9899,11.741667,45.938453,2000,2.838893
2,E20 East,8025.1,857.8,Lidingö,T-Centralen,8.0251,14.296667,33.679599,2000,2.331546
3,279 West,15364.8,1137.2,Vällingby,Bergshamra,15.3648,18.953333,48.639887,1200,1.055223
4,275 West,13454.3,1302.3,Vällingby,T-Centralen,13.4543,21.705000,37.192260,1200,0.921447
5,E20 South,8900.0,623.6,Skärholmen,Årstaberg,8.9000,10.393333,51.379089,2000,3.207184
6,75 Mid,4200.4,312.2,Årstaberg,Avicii Arena,4.2004,5.203333,48.435106,800,2.562460
7,73 South,9457.5,856.9,Farsta Shopping Centre,Avicii Arena,9.4575,14.281667,39.732758,800,0.933598
8,271 West,14464.8,1215.1,Skärholmen,Farsta Shopping Centre,14.4648,20.251667,42.855139,1200,0.987573
9,222 East,9847.3,855.9,Nacka Forum,T-Centralen,9.8473,14.265000,41.418717,1200,1.402033


In [144]:
# Create a copy of csv_data with swapped to_node and from_node columns
swapped_data = csv_data.copy()
swapped_data[['to_node', 'from_node']] = swapped_data[['from_node', 'to_node']]

# Append the swapped data to the original csv_data
#csv_data = pd.concat([csv_data, swapped_data], ignore_index=True)

# Display the first few rows of the updated DataFrame
#csv_data.head()

### Demand split

In [145]:
# Chat Gpt generated
# Construct the DataFrame
data = {
    "Node": [
        "Farsta Shopping Centre", "Skärholmen", "Vällingby", "Täby", "Lidingö",
        "Nacka Forum", "Sollentuna", "Bergshamra", "Avicii Arena", "Årstaberg", "T-Centralen"
    ],
    "Approx. Population": [
        60_523, 33_662, 71_042, 77_744, 48_162, 110_633, 76_237, 7_300, 70_779, 12_486, 40_312
    ],
    "Source": [
        "Farsta borough (2022 estimate)",
        "Skärholmen borough (2004)",
        "Hässelby-Vällingby borough (2014)",
        "Täby Municipality (31 Dec 2024)",
        "Lidingö urban area (2023)",
        "Nacka Municipality (2023)",
        "Sollentuna Municipality (2022)",
        "Bergshamra suburb (approx.)",
        "Östermalm borough (2014)",
        "Årsta locality (recent)",
        "Norrmalm district"
    ]
}

_df = pd.DataFrame(data)


In [146]:
import itertools

# Create a new DataFrame with demand for each Node from all other Nodes
demand_data = []

# Assuming not everyone works (Demographic)
_df["Approx. Population"] = (_df["Approx. Population"]/2).astype(int)
data_split = 0.9

for source, target in itertools.product(_df["Node"], repeat=2):
    source_population = _df.loc[_df["Node"] == source, "Approx. Population"].values[0]
    if target == "T-Centralen":
        demand = int(source_population * data_split)
    else:
        demand = int(source_population * (1 - data_split) / (_df.shape[0] - 1))  # Exclude T-Centralen
    demand_data.append({
        "Source": source,
        "Target": target,
        "Demand": demand
    })

demand_df = pd.DataFrame(demand_data)
demand_df["Demand"] = demand_df["Demand"].astype(int)
demand_df.head()

,Source,Target,Demand
0,Farsta Shopping Centre,Farsta Shopping Centre,302
1,Farsta Shopping Centre,Skärholmen,302
2,Farsta Shopping Centre,Vällingby,302
3,Farsta Shopping Centre,Täby,302
4,Farsta Shopping Centre,Lidingö,302


# Modeling

In [150]:
m = Container()

In [151]:
Nodes = Set(m, "Nodes", records=_df["Node"].unique().tolist())
Roads = Set(m, "Roads", records=csv_data["edge_name"].unique().tolist())

Nodes_ = Alias(m, "Noads_", Nodes)

In [152]:
Demands = Parameter(m, "demand", domain=[Nodes, Nodes_], description="How many from Source Node should go to Target Node", records=demand_df[["Source", "Target", "Demand"]])

Road_times = Parameter(m, "road_time", domain=[Roads], description="Time to travel on the road", records=csv_data[["edge_name", "duration_min"]])

GamspyException: Encountered data errors with symbol `road_time`. Possible causes are from duplicate records and/or domain violations. 

Use 'hasDuplicateRecords', 'findDuplicateRecords', 'dropDuplicateRecords', and/or 'countDuplicateRecords' to find/resolve duplicate records. 
Use 'hasDomainViolations', 'findDomainViolations', 'dropDomainViolations', and/or 'countDomainViolations' to find/resolve domain violations. 

GDX file was not created successfully.

In [ ]:
Road_utilization = Variable(m, "road", domain=[Nodes, Nodes_], description="Roads to be used")
